In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from src.predictionModule.LoadupSamples import LoadupSamples
from src.predictionModule.FilterSamples import FilterSamples
from src.predictionModule.MachineModels import MachineModels

import numpy as np
import pandas as pd
import polars as pl
import datetime

import logging
formatted_date = datetime.datetime.now().strftime("%d%b%y_%H%M").lower()

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(fmt="%(asctime)s - %(message)s")
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)
else:
    logger.handlers[:] = [handler]

#Output File handler
formatted_str = f"notebook-LGBMOnFiltered-optuna-{formatted_date}"
file_handler = logging.FileHandler(f"{formatted_str}.log", mode="w")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Usage
logger.info("This will print to the notebook's output cell")

2025-09-02 09:51:19,560 - This will print to the notebook's output cell


In [ ]:
params = {
    "idxAfterPrediction": 5,
    'timesteps': 35,
    'target_option': 'last',
    "LoadupSamples_time_scaling_stretch": True,
    "LoadupSamples_time_inc_factor": 61,
    
    "LSTM_units": 32,
    "LSTM_num_layers": 2,
    "LSTM_dropout": 1e-4,
    "LSTM_recurrent_dropout": 0.001157,
    "LSTM_learning_rate": 0.000046,
    "LSTM_optimizer": "adam",
    "LSTM_bidirectional": True,
    "LSTM_batch_size": 2**12,
    "LSTM_epochs": 4,
    "LSTM_l1": 0.000036,
    "LSTM_l2": 0.003569,
    "LSTM_inter_dropout": 0.052745,
    "LSTM_input_gaussian_noise": 0.001,
    "LSTM_conv1d": True,
    "LSTM_conv1d_kernel_size": 3,
    "LSTM_loss": "mse",

    "FilterSamples_q_up": 0.985033,
    "FilterSamples_days_to_train_end": 65,

    "FilterSamples_lincomb_epochs": 150,
    "FilterSamples_lincomb_lr": 0.0005,
    "FilterSamples_lincomb_probs_noise_std": 0.15,
    "FilterSamples_lincomb_show_progress": False,
    "FilterSamples_lincomb_subsample_ratio": 0.3,
    "FilterSamples_lincomb_sharpness": 2.3,
    "FilterSamples_lincomb_featureratio": 0.8,
    "FilterSamples_lincomb_itermax": 1,
    "FilterSamples_lincomb_init_toprand":  6,
    "FilterSamples_lincomb_batch_size": 2**12,

    "FilterSamples_cat_over20": True,
    "FilterSamples_cat_posOneYearReturn": False,
    "FilterSamples_cat_posFiveYearReturn": False,
    "FilterSamples_taylor_horizon_days": 65,
    "FilterSamples_taylor_roll_window_days": 10,
    "FilterSamples_taylor_weight_slope": 0.7,
    
    "LGB_num_boost_round": 950,
    "LGB_lambda_l1": 0.000100,
    "LGB_lambda_l2": 0.009786276249261908,
    "LGB_feature_fraction": 0.20813359498274574,
    "LGB_num_leaves": 191,
    "LGB_max_depth": 9,
    "LGB_learning_rate": 0.008855,
    "LGB_min_data_in_leaf": 350,
    "LGB_min_gain_to_split": 0.10066457576238419,
    "LGB_path_smooth": 0.5935679203578974,
    "LGB_min_sum_hessian_in_leaf": 0.3732876155751053,
    "LGB_max_bin": 850,
}

In [3]:
timegroup = "group_regOHLCV_over5years"
treegroup = "group_finanTo2011"

eval_date = datetime.date(year=2025, month=7, day=13)
evaldates = [eval_date - datetime.timedelta(days=i) for i in range(1, 6)]
start_train_date = datetime.date(year=2020, month=1, day=1)
split_Date = datetime.date(year=2025, month=1, day=1)
ls = LoadupSamples(
    train_start_date=start_train_date,
    test_dates=evaldates,
    treegroup=treegroup,
    timegroup=timegroup,
    params=params,
)
ls.load_samples(main_path = "../src/featureAlchemy/bin/")
ls.split_dataset(
    start_date=start_train_date,
    last_train_date=split_Date,
    last_test_date=eval_date
)

2025-09-02 09:52:09,832 - Test date 2025-07-12 not found in the database. Omitting.
2025-09-02 09:52:16,558 - Non-finite/too-large values in train y tree: 15 samples.
2025-09-02 09:52:16,562 - Non-finite/too-large values in train X tree: 86 samples.
2025-09-02 09:52:16,566 - Non-finite/too-large values in train y time: 15 samples.
2025-09-02 09:52:16,571 - Removing 101 samples from training data.


In [4]:
fs_pre = FilterSamples(
    Xtree_train=ls.train_Xtree,
    ytree_train=ls.train_ytree,
    treenames=ls.featureTreeNames,
    Xtree_test=ls.test_Xtree,
    ytree_test=ls.test_ytree,
    meta_train=ls.meta_pl_train,
    meta_test=ls.meta_pl_test,
    params=params
)

cat_mask_train, cat_mask_test = fs_pre.categorical_masks()
ls.apply_masks(cat_mask_train, cat_mask_test)

In [5]:
"""
Runs LSTM to generate feature(s) to add to the tree data.
"""
mm = MachineModels(params)

device = 'cuda'
starttime = datetime.datetime.now()
lstm_model, res_dict = mm.run_LSTM_torch(
    ls.train_Xtime, 
    ls.train_ytime, 
    ls.test_Xtime, 
    ls.test_ytime, 
    device=device
)
preds_train = mm.predict_LSTM_torch(lstm_model, ls.train_Xtime, batch_size=params["LSTM_batch_size"], device=device)
preds_test = mm.predict_LSTM_torch(lstm_model, ls.test_Xtime, batch_size=params["LSTM_batch_size"], device=device)
endtime = datetime.datetime.now()

logger.info(f"  LSTM RSME: {res_dict['val_rmse']*2/params['LoadupSamples_time_inc_factor']:.4f}")
logger.info(f"  LSTM completed in {endtime - starttime}.")
for q in [0.95, 0.98, 0.99]:
    q_pred = np.quantile(preds_train, q)
    logger.info(f"  LSTM quantile {q:.2f} of train predictions: {q_pred:.4f}")
    filtered = ls.train_ytree[preds_train >= q_pred]
    logger.info(f"  Result of quantile {q:.2f} of train set (mean): {filtered.mean():.4f}")

## Add LSTM predictions to the tree test set
train_std = np.std(preds_train)
test_std = 1.0 if np.std(preds_test) < 1e-6 else np.std(preds_test)
ls.train_Xtree = np.hstack((ls.train_Xtree, ((preds_train-1.0)/train_std).reshape(-1, 1)))
ls.test_Xtree = np.hstack((ls.test_Xtree, ((preds_test-1.0)/test_std).reshape(-1, 1)))
if isinstance(ls.featureTreeNames, list):
    ls.featureTreeNames.append("LSTM_Prediction")
elif isinstance(ls.featureTreeNames, np.ndarray):
    ls.featureTreeNames = np.append(ls.featureTreeNames, "LSTM_Prediction")

Epochs:   0%|          | 0/4 [00:00<?, ?it/s]

2025-09-02 09:53:04,433 - Epoch 1/4 — Train RMSE: 0.5143 — Validation RMSE: 0.4411


Epochs:  25%|██▌       | 1/4 [00:12<00:37, 12.41s/it]

2025-09-02 09:53:15,991 - Epoch 2/4 — Train RMSE: 0.4183 — Validation RMSE: 0.4087


Epochs:  50%|█████     | 2/4 [00:23<00:23, 11.91s/it]

2025-09-02 09:53:27,442 - Epoch 3/4 — Train RMSE: 0.4101 — Validation RMSE: 0.4078


Epochs:  75%|███████▌  | 3/4 [00:35<00:11, 11.70s/it]

2025-09-02 09:53:38,922 - Epoch 4/4 — Train RMSE: 0.4098 — Validation RMSE: 0.4076


Epochs: 100%|██████████| 4/4 [00:46<00:00, 11.73s/it]


2025-09-02 09:53:44,621 -   LSTM RSME: 0.0134
2025-09-02 09:53:44,622 -   LSTM completed in 0:01:00.534947.
2025-09-02 09:53:44,629 -   LSTM quantile 0.95 of train predictions: 0.5460
2025-09-02 09:53:44,631 -   Result of quantile 0.95 of train set (mean): 1.0005
2025-09-02 09:53:44,641 -   LSTM quantile 0.98 of train predictions: 0.5511
2025-09-02 09:53:44,642 -   Result of quantile 0.98 of train set (mean): 0.9991
2025-09-02 09:53:44,650 -   LSTM quantile 0.99 of train predictions: 0.5538
2025-09-02 09:53:44,651 -   Result of quantile 0.99 of train set (mean): 0.9982


In [6]:
import optuna
logger.setLevel(logging.INFO)

n_reruns = 8
time_delta_days = 25
split_dates = [split_Date - datetime.timedelta(days = time_delta_days * i) for i in range(0, n_reruns)]
n_testdays = 7
studytime = 60*60*2
n_startup_trials = 15
studyname = f"optuna_LGBMOnFiltered_{formatted_str}"

c:\Users\kimer\Desktop\RandomOdyssey\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
FilterSamples_method = "taylor"

def sample_params(trial: optuna.Trial):
    PARAMS_SPACE = {
        "LGB_num_boost_round": trial.suggest_int("LGB_num_boost_round", 750, 1500, step=50),
        "LGB_lambda_l1": trial.suggest_float("LGB_lambda_l1", 1e-6, 5e+1, log=True), #5e-2
        "LGB_lambda_l2": 1e-5,
        "LGB_feature_fraction": 0.7,
        "LGB_num_leaves": trial.suggest_int("LGB_num_leaves", 141, 211, step=10),
        "LGB_max_depth": trial.suggest_int("LGB_max_depth", 7, 10),
        "LGB_learning_rate": trial.suggest_float("LGB_learning_rate", 5e-3, 5e-1, log=True),
        "LGB_min_data_in_leaf": trial.suggest_int("LGB_min_data_in_leaf", 300, 500, step=50),
        "LGB_min_gain_to_split": 0.13,
        "LGB_path_smooth": 0.6,
        "LGB_min_sum_hessian_in_leaf": 1.0,
        "LGB_max_bin": trial.suggest_int("LGB_max_bin", 750, 950, step=50),

        "FilterSamples_method": "taylor",
        "FilterSamples_q_up": trial.suggest_float("FilterSamples_q_up", 0.985, 0.993)
    }
    return PARAMS_SPACE

def evaluate_objective(opt_params, trial_num):
    scores = []
    for sd in split_dates[:n_reruns]:
        logger.info(f"Trial number {trial_num}")
        lsc = ls.copy(deep=True)
        last_test_day = sd + datetime.timedelta(days=n_testdays)
        lsc.split_dataset(start_date=start_train_date, last_train_date=sd, last_test_date=last_test_day)

        fs = FilterSamples(
            Xtree_train=lsc.train_Xtree, ytree_train=lsc.train_ytree, treenames=lsc.featureTreeNames,
            Xtree_test=lsc.test_Xtree,  ytree_test=lsc.test_ytree,
            meta_train=lsc.meta_pl_train, meta_test=lsc.meta_pl_test, params=opt_params
        )
        if opt_params["FilterSamples_method"] == "taylor":
            mask_train, mask_test = fs.taylor_feature_masks()
        if opt_params["FilterSamples_method"] == "lincomb":
            mask_train, mask_test = fs.lincomb_masks()
            
        # LGB model
        mm = MachineModels(opt_params)
        startTime  = datetime.datetime.now()
        lgb_model, lgb_res_dict = mm.run_LGB(
            X_train=lsc.train_Xtree[mask_train],
            y_train=lsc.train_ytree[mask_train],
            X_test=lsc.test_Xtree[mask_test],
            y_test=lsc.test_ytree[mask_test]
        )
        logger.info(f"  LGB completed in {datetime.datetime.now() - startTime}.")
        
        # LGB Predictions
        startTime  = datetime.datetime.now()
        best_iter = getattr(lgb_model, "best_iteration", None)
        y_train_pred_masked = lgb_model.predict(lsc.train_Xtree[mask_train], num_iteration=best_iter)
        y_test_pred_masked  = lgb_model.predict(lsc.test_Xtree[mask_test],  num_iteration=best_iter)
        rmse_train = np.sqrt(np.mean((y_train_pred_masked - lsc.train_ytree[mask_train]) ** 2))
        rmse_test = np.sqrt(np.mean((y_test_pred_masked - lsc.test_ytree[mask_test]) ** 2))
        logger.info(f"  Train (lgbm_pred - ytree)       -> RMSE: {rmse_train:.4f}")
        logger.info(f"  Test (lgbm_pred - ytree)        -> RMSE: {rmse_test:.4f}")
        logger.info(f"  LGB Prediction completed in {datetime.datetime.now() - startTime}.")
        
        m = 5
        meta_pl = lsc.meta_pl_test.filter(pl.Series(mask_test)).with_columns(
            pl.Series("prediction_ratio", y_test_pred_masked)
        )
        meta_pl_filtered = (
            meta_pl
            .sort(["date", "prediction_ratio"], descending=[False, True])
            .with_columns(
                pl.col("prediction_ratio")
                .rank(method="random", descending=True)
                .over("date")
                .alias("prediction_rank")
            ).filter(pl.col("prediction_rank") <= m)
        )
        agg_exprs = [
            pl.col("prediction_ratio").max().alias("max_pred"),  # this is also .first()
            pl.col("prediction_ratio").log().mean().exp().alias("mean_pred"),
            pl.col("target_ratio").log().mean().exp().alias("mean_res"),
            pl.col("target_ratio")
                .sort_by(pl.col("prediction_ratio"), descending=True)
                .first()
                .alias("top_res"),
            pl.len().alias("n_entries"),
        ]
        test_df_perdate = meta_pl_filtered.group_by("date").agg(agg_exprs).sort("date")
        
        if test_df_perdate.height == 0:
            pred_meanlast = pred_toplast = res_meanlast = res_toplast = 1.0
        else:
            pred_meanlast = test_df_perdate["mean_pred"].item(-1)
            pred_toplast  = test_df_perdate["max_pred"].item(-1)
            res_meanlast  = test_df_perdate["mean_res"].item(-1)
            res_toplast   = test_df_perdate["top_res"].item(-1)
        res_sum_n = int(test_df_perdate["n_entries"].sum())
        
        score = res_meanlast

        # Final  Analysis
        fs_score_train = fs.evaluate_mask(mask_train, lsc.meta_pl_train['date'], lsc.train_ytree)
        fs_score_test  = fs.evaluate_mask(mask_test,  lsc.meta_pl_test['date'],  lsc.test_ytree)
        logger.info(f"  FilterSamples Score (train) = {fs_score_train}")
        logger.info(f"  FilterSamples Score (test)  = {fs_score_test}")
        logger.info(f"  Final top last prediction ratio: {pred_toplast:.4f}")
        logger.info(f"  Final last mean prediction ratio: {pred_meanlast:.4f}")
        logger.info(f"  Final top last P/L Ratio: {res_toplast:.4f}")
        logger.info(f"  Final mean last P/L Ratio: {res_meanlast:.4f}")
        logger.info(f"  Number of entries: {res_sum_n}")

        scores.append(score)

    s = np.array(scores, dtype=float)
    s[~np.isfinite(s)] = 1.0
    s[s <= 0] = 1.0
    return float(np.log(s).mean()) if s.size else float("-inf")

def make_objective():
    def objective(trial: optuna.Trial) -> float:
        opt_params = params.copy()
        opt_params.update(sample_params(trial))
        return evaluate_objective(opt_params, trial.number)
    return objective

In [8]:
optuna.logging.enable_propagation()
sampler = optuna.samplers.TPESampler(n_startup_trials=n_startup_trials)
study = optuna.create_study(
    study_name = studyname,
    storage="sqlite:///sandbox_optuna.db",
    direction="maximize",
    load_if_exists=True,
    sampler=sampler,
)
study.optimize(make_objective(), timeout=studytime)

logger.info(f"Best parameters: {study.best_params}")
logger.info(f"Best score: {study.best_value}")

df: pd.DataFrame = study.trials_dataframe()
logger.info("\nTrials DataFrame:")
logger.info(df.sort_values("value").to_string())

param_importances = optuna.importance.get_param_importances(study)
logger.info("Parameter Importances:")
for key, value in param_importances.items():
    logger.info(f"{key}: {value}")

[I 2025-09-01 14:58:31,223] A new study created in RDB with name: optuna_LGBMOnFiltered_notebook-LGBMOnFiltered-optuna-01sep25_1455


2025-09-01 14:58:31,223 - A new study created in RDB with name: optuna_LGBMOnFiltered_notebook-LGBMOnFiltered-optuna-01sep25_1455
2025-09-01 14:58:31,274 - Trial number 0
2025-09-01 14:58:44,783 - FilterSamples/Taylor: (train) mean of y values 1.0006264340729107.
2025-09-01 14:58:44,792 - FilterSamples/Taylor: (test) mean of y values 1.0022854508909003.
2025-09-01 14:58:45,365 - FilterSamples/Taylor: Selected feature 'FeatureTA_volatility_kcc' with score 1.232184 and threshold 1.06326 (q=0.9921649092960316).
2025-09-01 14:58:46,054 - FilterSamples/Taylor: Final score (train): 0.9977
2025-09-01 14:58:46,055 - FilterSamples/Taylor: Final score (test): 0.9919
2025-09-01 14:58:46,057 - FilterSamples/Taylor: Fraction of days with no coverage by test mask: 0.00%
2025-09-01 14:58:46,655 - Iteration 0: train's rmse: 0.15153023130758622, valid's rmse: 0.05398200372039453
2025-09-01 14:58:46,883 - Iteration 50: train's rmse: 0.1324053153839176, valid's rmse: 0.05443174226088282
2025-09-01 14:58:

[I 2025-09-01 15:00:45,075] Trial 0 finished with value: -0.009654044922632046 and parameters: {'LGB_num_boost_round': 1300, 'LGB_lambda_l1': 2.5838184206442354, 'LGB_num_leaves': 211, 'LGB_max_depth': 10, 'LGB_learning_rate': 0.005841516723721373, 'LGB_min_data_in_leaf': 300, 'LGB_max_bin': 800, 'FilterSamples_q_up': 0.9921649092960316}. Best is trial 0 with value: -0.009654044922632046.


2025-09-01 15:00:45,075 - Trial 0 finished with value: -0.009654044922632046 and parameters: {'LGB_num_boost_round': 1300, 'LGB_lambda_l1': 2.5838184206442354, 'LGB_num_leaves': 211, 'LGB_max_depth': 10, 'LGB_learning_rate': 0.005841516723721373, 'LGB_min_data_in_leaf': 300, 'LGB_max_bin': 800, 'FilterSamples_q_up': 0.9921649092960316}. Best is trial 0 with value: -0.009654044922632046.
2025-09-01 15:00:45,145 - Trial number 1
2025-09-01 15:00:59,188 - FilterSamples/Taylor: (train) mean of y values 1.0006264340729107.
2025-09-01 15:00:59,188 - FilterSamples/Taylor: (test) mean of y values 1.0022854508909003.
2025-09-01 15:00:59,683 - FilterSamples/Taylor: Selected feature 'FeatureTA_volatility_ui' with score 1.275523 and threshold 2.15499 (q=0.9911241400671059).
2025-09-01 15:01:00,375 - FilterSamples/Taylor: Final score (train): 0.9957
2025-09-01 15:01:00,378 - FilterSamples/Taylor: Final score (test): 1.0321
2025-09-01 15:01:00,380 - FilterSamples/Taylor: Fraction of days with no cov

[I 2025-09-01 15:02:57,964] Trial 1 finished with value: -0.005979853037950303 and parameters: {'LGB_num_boost_round': 800, 'LGB_lambda_l1': 0.0010357468784301484, 'LGB_num_leaves': 211, 'LGB_max_depth': 9, 'LGB_learning_rate': 0.012933665972419878, 'LGB_min_data_in_leaf': 500, 'LGB_max_bin': 850, 'FilterSamples_q_up': 0.9911241400671059}. Best is trial 1 with value: -0.005979853037950303.


...

2025-09-01 16:55:10,264 - Trial 52 finished with value: 0.011097057292175277 and parameters: {'LGB_num_boost_round': 1100, 'LGB_lambda_l1': 0.0022626118666894024, 'LGB_num_leaves': 211, 'LGB_max_depth': 7, 'LGB_learning_rate': 0.03975831650586436, 'LGB_min_data_in_leaf': 400, 'LGB_max_bin': 850, 'FilterSamples_q_up': 0.9860038537103262}. Best is trial 40 with value: 0.018251351477821552.
2025-09-01 16:55:10,345 - Trial number 53
2025-09-01 16:55:23,522 - FilterSamples/Taylor: (train) mean of y values 1.0006264340729107.
2025-09-01 16:55:23,522 - FilterSamples/Taylor: (test) mean of y values 1.0022854508909003.
2025-09-01 16:55:24,058 - FilterSamples/Taylor: Selected feature 'FeatureTA_volatility_ui' with score 1.231216 and threshold 1.82342 (q=0.9854692226417241).
2025-09-01 16:55:25,052 - FilterSamples/Taylor: Final score (train): 0.9968
2025-09-01 16:55:25,055 - FilterSamples/Taylor: Final score (test): 1.0214
2025-09-01 16:55:25,055 - FilterSamples/Taylor: Fraction of days with no c

[I 2025-09-01 16:57:23,696] Trial 53 finished with value: 0.017187099760488406 and parameters: {'LGB_num_boost_round': 1200, 'LGB_lambda_l1': 0.0003173437826472909, 'LGB_num_leaves': 211, 'LGB_max_depth': 7, 'LGB_learning_rate': 0.02675755144616763, 'LGB_min_data_in_leaf': 400, 'LGB_max_bin': 850, 'FilterSamples_q_up': 0.9854692226417241}. Best is trial 40 with value: 0.018251351477821552.


2025-09-01 16:57:23,696 - Trial 53 finished with value: 0.017187099760488406 and parameters: {'LGB_num_boost_round': 1200, 'LGB_lambda_l1': 0.0003173437826472909, 'LGB_num_leaves': 211, 'LGB_max_depth': 7, 'LGB_learning_rate': 0.02675755144616763, 'LGB_min_data_in_leaf': 400, 'LGB_max_bin': 850, 'FilterSamples_q_up': 0.9854692226417241}. Best is trial 40 with value: 0.018251351477821552.
2025-09-01 16:57:23,771 - Trial number 54
2025-09-01 16:57:37,084 - FilterSamples/Taylor: (train) mean of y values 1.0006264340729107.
2025-09-01 16:57:37,084 - FilterSamples/Taylor: (test) mean of y values 1.0022854508909003.
2025-09-01 16:57:37,593 - FilterSamples/Taylor: Selected feature 'FeatureTA_volatility_ui' with score 1.232440 and threshold 1.81268 (q=0.9850634845622274).
2025-09-01 16:57:38,338 - FilterSamples/Taylor: Final score (train): 0.9971
2025-09-01 16:57:38,342 - FilterSamples/Taylor: Final score (test): 1.0193
2025-09-01 16:57:38,344 - FilterSamples/Taylor: Fraction of days with no c

[I 2025-09-01 16:59:35,402] Trial 54 finished with value: 0.008079498313608081 and parameters: {'LGB_num_boost_round': 1250, 'LGB_lambda_l1': 0.0008169006410172465, 'LGB_num_leaves': 211, 'LGB_max_depth': 7, 'LGB_learning_rate': 0.060916147176329635, 'LGB_min_data_in_leaf': 450, 'LGB_max_bin': 900, 'FilterSamples_q_up': 0.9850634845622274}. Best is trial 40 with value: 0.018251351477821552.


2025-09-01 16:59:35,402 - Trial 54 finished with value: 0.008079498313608081 and parameters: {'LGB_num_boost_round': 1250, 'LGB_lambda_l1': 0.0008169006410172465, 'LGB_num_leaves': 211, 'LGB_max_depth': 7, 'LGB_learning_rate': 0.060916147176329635, 'LGB_min_data_in_leaf': 450, 'LGB_max_bin': 900, 'FilterSamples_q_up': 0.9850634845622274}. Best is trial 40 with value: 0.018251351477821552.
2025-09-01 16:59:35,404 - Best parameters: {'LGB_num_boost_round': 850, 'LGB_lambda_l1': 34.67228607263477, 'LGB_num_leaves': 191, 'LGB_max_depth': 7, 'LGB_learning_rate': 0.09603141855741754, 'LGB_min_data_in_leaf': 300, 'LGB_max_bin': 950, 'FilterSamples_q_up': 0.9892718192209784}
2025-09-01 16:59:35,411 - Best score: 0.018251351477821552
2025-09-01 16:59:35,419 - 
Trials DataFrame:
2025-09-01 16:59:35,426 -     number     value             datetime_start          datetime_complete               duration  params_FilterSamples_q_up  params_LGB_lambda_l1  params_LGB_learning_rate  params_LGB_max_bin  

In [9]:
df.to_parquet(f"{formatted_str}.parquet", index=False)

In [10]:
df = pl.read_parquet("notebook-LGBMOnFiltered-optuna-30aug25_1713.parquet")

In [11]:
df_roll_mean = df.sort("value").select(
    [pl.col("value")] + 
    [pl.col(c).rolling_mean(window_size=10).alias(f"{c}_rollmean10") for c in df.columns if c.startswith("params_")]
)